In [1]:
import chess
import RL_utils

In [2]:
# Our existing CNN model
model_path = "models/TORCH_250EPOCH 1.pth"

## Load stockfish

In [3]:
from RL_utils import PositionEvaluator

In [4]:
stockfish_path = "models/stockfish/stockfish-windows-x86-64-avx2.exe"
evaluator = PositionEvaluator(stockfish_path, elo_rating=1400)

Stockfish initialized with ELO: 1400


In [5]:
board = chess.Board()
board.push_uci("e2e4")
board.push_uci("h7h6")

Move.from_uci('h7h6')

In [6]:
eval_score = evaluator.evaluate_position(board)
print(f"Position evaluation: {eval_score}")

Position evaluation: 0.83


In [7]:
best_moves = evaluator.get_best_moves(board, 3)
print(f"Best moves: {best_moves}")

Best moves: [{'Move': 'd2d4', 'Centipawn': 83, 'Mate': None}, {'Move': 'g1f3', 'Centipawn': 71, 'Mate': None}, {'Move': 'b1c3', 'Centipawn': 68, 'Mate': None}]


## Load our CNN trained model

In [8]:
model, device = RL_utils.load_model(model_path)
model = model.to(device)

In [9]:
import torch
dummy_input = torch.randn(1, 19, 8, 8).to(device)
with torch.no_grad():
    output = model(dummy_input)
    print(f"Model output shape: {output.shape}")  # Should be [1, 4288]
    print("Model loaded successfully!")

Model output shape: torch.Size([1, 4288])
Model loaded successfully!


## Load the training data

In [11]:
pgn_file = "games/lichess_db_2016-04.pgn"
data = []
data += RL_utils.extract_middlegame_positions(pgn_file, evaluator, num_positions=10)
print(f"Total positions loaded: {len(data)}")

Extracted 10 middle game positions
Total positions loaded: 10


In [12]:
# Test creating a batch
if data:
    board_tensors, legal_masks, boards = RL_utils.create_training_batch(data, batch_size=4)
    print(f"Batch shapes:")
    print(f"  Board tensors: {board_tensors.shape}")
    print(f"  Legal masks: {legal_masks.shape}")
    print(f"  Number of boards: {len(boards)}")
    
    # Show a sample position
    print(f"\nSample position FEN: {boards[0].fen()}")

Batch shapes:
  Board tensors: torch.Size([4, 19, 8, 8])
  Legal masks: torch.Size([4, 4288])
  Number of boards: 4

Sample position FEN: rn3rk1/ppp1b1pp/1n3p2/4p3/4P1b1/2P2NP1/PP3PBP/R1BR1NK1 w - - 1 13


In [16]:
boards[0]
eval_score = evaluator.evaluate_position(boards[0])
print(f"Position evaluation: {eval_score}")

Position evaluation: 0.11


In [35]:
engine.quit()

NameError: name 'engine' is not defined